In [1]:
import pandas as pd
import numpy as np
import warnings
import sys

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

# Film Clean

Para el tratamiento de esta tabla realizaremos varios procesos:

**Índice**

1. [En primer lugar aplicaremos funciones a la columna 'title' para mejorar su legibilidad](#id1)

2. [Cambiaremos el nombre de release_year, entendiendo que el dato recibido es la fecha de adquisición](#id2)

3. [Eliminación de la columna 'original_language_id' debido a que esta compuesta íntegramente por nulos](#id3)

4. [Observación de la columna 'rating' buscando unificar las categorias](#id4)

5. [Separación de la columna 'special_features', buscando la creación de nuevas columnas con valores booleanos](#id5)

6. [Eliminación de la columna 'last_update' dedido a que únicamente contiene un valor](#id6)

7. [Acortamiento de la descripción del filme](#id7)

In [2]:
df = pd.read_csv('../raw_data/film.csv')
df.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,NaN,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,NaN,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,NaN,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,NaN,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,NaN,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
film_id,1000.0,500.500,288.819436,1.00,250.75,500.50,750.25,1000.00
release_year,1000.0,2006.000,0.000000,2006.00,2006.00,2006.00,2006.00,2006.00
language_id,1000.0,1.000,0.000000,1.00,1.00,1.00,1.00,1.00
original_language_id,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rental_duration,1000.0,4.985,1.411654,3.00,4.00,5.00,6.00,7.00
rental_rate,1000.0,2.980,1.646393,0.99,0.99,2.99,4.99,4.99
length,1000.0,115.272,40.426332,46.00,80.00,114.00,149.25,185.00
replacement_cost,1000.0,19.984,6.050833,9.99,14.99,19.99,24.99,29.99


In [4]:
df.isnull().sum()

film_id                    0
title                      0
description                0
release_year               0
language_id                0
original_language_id    1000
rental_duration            0
rental_rate                0
length                     0
replacement_cost           0
rating                     0
special_features           0
last_update                0
dtype: int64

<div id='id1' />

# 1. Limpieza 'Title'

In [5]:
df.title = df.title.apply(lambda x: x.lower())
df.title = df.title.apply(lambda x: x.title())

<div id='id2' />

# 2. Limpieza 'release_year'

In [6]:
df = df.rename(columns={'release_year': 'adquisition_year'})

<div id='id3' />

# 3. Limpieza 'original_language_id'

In [7]:
df = df.drop('original_language_id', axis=1)

<div id='id4' />

# 4. Limpieza 'rating'

In [8]:
df.rating.unique()

array(['PG', 'G', 'NC-17', 'PG-13', 'R'], dtype=object)

observamos que la columna no tiene valores fuera de lugar. Simplemente dejamos una leyenda para mayor información sobre los códigos:
- 'G': público general
- 'PG': Se recomienda orientación de los padres
- 'PG-13': Se advierte enfáticamente a los padres
- 'R': Restringifo, solo mayores de 18 años 
- 'NC-17': Solo para adultos

<div id='id5' />

# 5. Limpieza 'special_features'

In [9]:
sys.path.append('../functions')

from clean_functions import *

In [10]:
# Entrada y salida de funciones explicado en '../functions/clean_functions.py'
df['trailer'] = df.special_features.apply(trailer)
df['commentaries'] = df.special_features.apply(commentaries)
df['behind_scenes'] = df.special_features.apply(behind)
df['deleted_scenes'] = df.special_features.apply(deleted)
df = df.drop('special_features', axis=1)

<div id='id6' />

# 6. Limpieza 'last_update'

In [11]:
df = df.drop('last_update', axis=1)

<div id='id7' />

# 7. Limpieza 'description'

In [12]:
#Aplicamos la función size guardada en '../functions/clean_functions.py'
df.description = df.description.apply(size)

### Exportación

In [13]:
df.to_csv('../cleaned_data/3_clean_film.csv' , index=False)